In [3]:
# Import relevant functionality
import os
import asyncio
import requests
import urllib.parse
import json
import configparser
import datetime
from datetime import date
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from msgraph.generated.models.o_data_errors.o_data_error import ODataError
from configparser import SectionProxy
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_openai import OpenAI
from langchain_core.runnables import ConfigurableField
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv, find_dotenv
from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from langgraph.types import interrupt
from msgraph.generated.users.users_request_builder import UsersRequestBuilder
from kiota_abstractions.base_request_configuration import RequestConfiguration

# Graph permissions scopes
# For multiple scopes, separate with a space
scopes = 'user.read Calendars.Read Calendars.ReadBasic Calendars.Read Calendars.ReadWrite'

#load the environment
load_dotenv("C:\\Users\\sakov\\bootcamp\\LangChain Agents\\.env", override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
TENANT_ID = os.getenv("TENANT_ID")


@tool
async def getcontacts():
    """Get a list of user contacts and display with their full name, office location, email id, company name"""
    print("Ooh! I need a Microsoft tool to answer this query! You might need to login!","\n")
    credential = DeviceCodeCredential(CLIENT_ID, tenant_id = TENANT_ID)
    client = GraphServiceClient(credential, scopes.split(' '))
    # To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python
    query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
		top = "10",
    )
    request_configuration = RequestConfiguration(
        query_parameters = query_params,
    )
    contacts = await client.me.contacts.get(request_configuration = request_configuration)
    #contacts = await client.me.contacts.get()
    #print("Hello, ",contacts)
    return contacts


@tool
async def getevents():
    """Get a list of my events from my calendar"""
    print("Ooh! I need a Microsoft tool to answer this query! You might need to login!","\n")
    credential = DeviceCodeCredential(CLIENT_ID, tenant_id = TENANT_ID)
    client = GraphServiceClient(credential, scopes.split(' '))
    # To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python
    query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
		top = "0",
    )
    request_configuration = RequestConfiguration(
        query_parameters = query_params,
    )
    events = await client.me.calendar.events.get(request_configuration = request_configuration)
    #events = await client.me.calendar.events.get()
    return events
    
@tool
def getweather(city: str, key: str) -> str:
    """Get the current and the future weather conditions for the city provided by the user"""
    # base_url variable to store url
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    #get access token
    complete_url = base_url + "appid=" + OPENWEATHERMAP_API_KEY + "&q=" + city
    response = requests.get(complete_url)
    x = response.json()
    print("\n The temperature in "+city+" is "+str(x["main"]["temp"]))
    return x

#get current date for system context
#today = date.today()
#print(today)
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who can talk to the open weather map APIs to get the current weather of any city. Your name is Clippy. Today's date is 2025-01-10. Please format the tool responses you get into much more readable format. You can fetch their contacts. You can search the web for anything else you don't understand. You can get their calendar events. You can also give information to users about the holidays in their regions. Their DTO policies. Their timeoff policies. Anything they want to know about their employee benefits. The access_tokens are available in the tools so don't ask the users again."), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

search = TavilySearchResults(max_results=2)

tools = [getweather, getcontacts, getevents, search]

llm = ChatOpenAI(model="gpt-4o")

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
async def call_model(state: MessagesState):
    prompt = prompt.invoke(state)
    response = model.ainvoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = ChatMessageHistory(session_id="test-session")
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "def234"}}

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, trim_intermediate_steps=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("Hello! I am your personal companion, Clippy!. Ask me about your meetings. Or the weather. I can answer most things about the world!","\n")
query="Hey Clippy! I am Sai Deepthi Kovvuru. I am travelling to vancouver tomorrow. Do I have any important meetings tomorrow?"
print("------------------")
print("User prompt: ",query)

await agent_with_chat_history.ainvoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)



query ="Alright. In that case, I will proceed with my travel. Are there any major sports events happening in Vancover tomorrow? I do not want to get stuck in traffic."
print("-----------------------------------------------------------------------------------------------","\n")
print("User prompt: ",query)

agent_with_chat_history.invoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)

query="Okay. I will keep my travel on then! Is there anyone in my contacts who is in the canada office location of Montoso, that I can meet?"
print("----------------------------------------------------------------------------------------------","\n")
print("User prompt: ",query)

await agent_with_chat_history.ainvoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)

query="I might want to start today. What is the weather like, in Vancouver tonight?"
print("----------------------------------------------------------------------------------------------","\n")
print("User prompt: ",query)

agent_with_chat_history.invoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)

Hello! I am your personal companion, Clippy!. Ask me about your meetings. Or the weather. I can answer most things about the world! 

------------------
User prompt:  Hey Clippy! I am Sai Deepthi Kovvuru. I am travelling to vancouver tomorrow. Do I have any important meetings tomorrow?


> Entering new AgentExecutor chain...

Invoking: `getevents` with `{}`


Ooh! I need a Microsoft tool to answer this query! You might need to login! 

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E6HUBJ44L to authenticate.
EventCollectionResponse(additional_data={'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#users('2be45935-77e0-45e6-a816-42e5966c0b1c')/calendar/events"}, odata_count=None, odata_next_link=None, value=[])Hi Sai Deepthi Kovvuru! You don't have any important meetings scheduled for tomorrow, so you can enjoy your trip to Vancouver without any worries about missing an event. If you have any other questions or need furthe

{'input': 'I might want to start today. What is the weather like, in Vancouver tonight?',
 'chat_history': [HumanMessage(content='Hey Clippy! I am Sai Deepthi Kovvuru. I am travelling to vancouver tomorrow. Do I have any important meetings tomorrow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi Sai Deepthi Kovvuru! You don't have any important meetings scheduled for tomorrow, so you can enjoy your trip to Vancouver without any worries about missing an event. If you have any other questions or need further assistance, feel free to ask!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Alright. In that case, I will proceed with my travel. Are there any major sports events happening in Vancover tomorrow? I do not want to get stuck in traffic.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='It seems there are no major sports events specifically happening in Vancouver on January 11, 2025, that would likely cause significant t